In [1]:
library(tidyverse)
library(ape)
library(microbiome)
library(ggthemes)     # additional themes fro ggplot2
library(ggpubr)
library(vegan)
library(repr)
library(ggpmisc)      # to use stat_poly_eq
library(RColorBrewer) # nice color options
library(gridExtra)    # gridding plots
library(viridis)
library(ggrepel)
#library(wesanderson) #new palettes http://www.sthda.com/english/wiki/ggplot2-colors-how-to-change-colors-automatically-and-manually
library(rioja)        # plotting poackages for tabular bubbleplots
library(reshape2) 
library(dada2)
library(DECIPHER)
#library(ggtern)
library(dplyr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘ape’


The following object is masked from ‘package:dplyr’:

    where


Loading required package: phyloseq


microbiome R package (microbiome.github.com)
    


 Copyright (C) 2011-2022 Leo Lahti, 
    Sudarshan Shetty et al. <microbiome.github.io>



Attaching package: ‘microbiome’


The following object is masked from ‘package:ggplot2’:

    alpha


The following object is masked from ‘package:base’:

    transform



Attaching package: ‘ggpubr’




In [2]:
options(repr.plot.width=12, repr.plot.height=8)
set.seed(10000)

theme_glab <- function(base_size = 20,
                    base_family = "",
                    base_line_size = base_size / 180,
                    base_rect_size = base_size / 180) {
   
    font <- "Helvetica" #assign font family up front
   
    theme_bw(base_size = base_size,
                base_family = base_family,
                base_line_size = base_line_size) %+replace%
    theme(
        legend.background =  element_blank(),
        legend.title =       element_text(color = rgb(100, 100, 100, maxColorValue = 255),
                                          size = rel(0.65),
                                         hjust = 0),
        legend.text =        element_text(color = rgb(100, 100, 100, maxColorValue = 255),
                                          size = rel(0.65)),
        legend.key.size =    unit(0.8, "lines"),
     
      plot.title = element_text(
        color = rgb(100, 100, 100, maxColorValue = 255),
        hjust = 0),
       
      axis.title = element_text(
        color = rgb(100, 100, 100, maxColorValue = 255),
        size = rel(0.65)),
      axis.text = element_text(
        color = rgb(100, 100, 100, maxColorValue = 255),
        size = rel(0.65)),
       
      plot.caption = element_text(
        color = rgb(100, 100, 100, maxColorValue = 255),
        size = rel(0.7),
        hjust = 1),
       
      panel.grid.major = element_blank(),  
      panel.grid.minor = element_blank(),  
      panel.border = element_rect(fill = NA, colour = rgb(100, 100, 100, maxColorValue = 255)),

     
      complete = TRUE
    )
}


In [3]:
CAs_otu_raw <- read.csv("dataset_CoEvolve/Coevolve_distribution_bacteria.csv", sep="\t", header=TRUE)

CAs_otu_raw

Expedition,hmm,CA_class,Total.coverage,Normalized_total_coverage,Sample
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
arg19,Alpha_CA_prok,Alpha,18.1284,0.62942824,AG11
arg19,Alpha_CA_prok,Alpha,16.0836,0.40440682,AG19
arg19,Alpha_CA_prok,Alpha,58.5673,0.67315113,AG1
arg19,Alpha_CA_prok,Alpha,14.7281,0.33881677,AG3
arg19,Alpha_CA_prok,Alpha,105.3205,2.25344190,AO190224-f
arg19,Alpha_CA_prok,Alpha,1249.2500,115.77900293,BJ190227-f
arg19,Alpha_CA_prok,Alpha,162.7612,5.77549173,PG190225-f
arg19,Alpha_CA_prok,Alpha,115.6349,2.84516220,PM190223-f
arg19,Alpha_CA_prok,Alpha,17.5827,0.52853697,VV190228-f


In [4]:
subset1 <- as.data.frame(CAs_otu_raw[, c("hmm", "Normalized_total_coverage", "Sample")])
subset2 <- subset1[!duplicated(subset1),]
otu_table <- as.data.frame(subset2 %>% pivot_wider(names_from = Sample, 
                                                   values_from = Normalized_total_coverage, 
                                                   values_fill = list(Value = 0)))
row.names(otu_table) <- otu_table$hmm
otu_table$hmm <- NULL


otu_table

,AG11,AG19,AG1,AG3,AO190224-f,BJ190227-f,PG190225-f,PM190223-f,VV190228-f,BM200304F,⋯,AG5,CR200310F,EM200307F,EM200307S,BQF,BQ,BQS,f15,GN-2-F,KR-2-S
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Alpha_CA_prok,0.6294282,0.4044068,0.6731511,0.3388168,2.2534419,115.7790,5.775492,2.845162,0.5285370,5.541804,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Beta_CA_prok,2.1759689,2.4111634,1.2723821,9.6167361,5.2913761,168.3473,5.518066,5.171825,1.8757987,6.388322,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Epsilon_CA_prok,7.6768135,NA,0.2304002,NA,0.4676319,NA,10.703583,1.168960,0.8424151,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Gamma_CA_prok,7.0067657,2.5975411,4.1934433,13.7333319,12.9424084,135.1830,4.368973,26.054900,5.0216619,7.002997,⋯,8.521976,29.0798,0.3312554,1.443318,209.1669,46.49564,352.617,177.4402,88.55266,72.40057


In [5]:
otu_table_final <- as.data.frame(otu_table)
otu_table_final[is.na(otu_table_final)] <- 0

otu_table_final

,AG11,AG19,AG1,AG3,AO190224-f,BJ190227-f,PG190225-f,PM190223-f,VV190228-f,BM200304F,⋯,AG5,CR200310F,EM200307F,EM200307S,BQF,BQ,BQS,f15,GN-2-F,KR-2-S
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Alpha_CA_prok,0.6294282,0.4044068,0.6731511,0.3388168,2.2534419,115.7790,5.775492,2.845162,0.5285370,5.541804,⋯,0.000000,0.0000,0.0000000,0.000000,0.0000,0.00000,0.000,0.0000,0.00000,0.00000
Beta_CA_prok,2.1759689,2.4111634,1.2723821,9.6167361,5.2913761,168.3473,5.518066,5.171825,1.8757987,6.388322,⋯,0.000000,0.0000,0.0000000,0.000000,0.0000,0.00000,0.000,0.0000,0.00000,0.00000
Epsilon_CA_prok,7.6768135,0.0000000,0.2304002,0.0000000,0.4676319,0.0000,10.703583,1.168960,0.8424151,0.000000,⋯,0.000000,0.0000,0.0000000,0.000000,0.0000,0.00000,0.000,0.0000,0.00000,0.00000
Gamma_CA_prok,7.0067657,2.5975411,4.1934433,13.7333319,12.9424084,135.1830,4.368973,26.054900,5.0216619,7.002997,⋯,8.521976,29.0798,0.3312554,1.443318,209.1669,46.49564,352.617,177.4402,88.55266,72.40057


In [6]:
tax_table_raw  <- as.data.frame(CAs_otu_raw[, c("hmm", "CA_class")])
tax_table <- tax_table_raw[!duplicated(tax_table_raw), ]
row.names(tax_table) <- tax_table$hmm

tax_table

tax_table <- as.matrix(tax_table)

,hmm,CA_class
,<chr>,<chr>
Alpha_CA_prok,Alpha_CA_prok,Alpha
Beta_CA_prok,Beta_CA_prok,Beta
Epsilon_CA_prok,Epsilon_CA_prok,Epsilon
Gamma_CA_prok,Gamma_CA_prok,Gamma


In [7]:
sample_data <- read.csv("dataset_CoEvolve/Coevolve_env_data.csv", row.names=1)
sample_data_final <-as.data.frame(sample_data)
sample_data_final

,expedition,nation,site_name,latitude,longitude,type,temperature,pH,C13,dissolved_oxygen,salinity,alkalinity,spc
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AG1,ARG19,Argentina,Incachule,-24.282129,-66.46676,S,46.90,6.52,NA,10.0,1.54,NA,3000.0
AG11,ARG19,Argentina,Pastos Grandes,-24.364589,-66.57113,S,44.90,8.74,-15.33,50.0,0.61,15.400,1288.0
AG13,ARG19,Argentina,Galán Fumaroles (El Diablo),-25.858188,-66.99269,S,80.00,7.75,-14.81,10.0,0.19,NA,429.0
AG15,ARG19,Argentina,Galán Fumaroles (El Diablo),-25.858243,-66.99282,S,80.00,3.21,-12.40,60.0,1.02,21.120,2050.0
AG17,ARG19,Argentina,Galán La Colcha,-26.032911,-66.98609,BG,84.00,6.94,-2.62,20.0,6.10,300.080,10618.0
AG19,ARG19,Argentina,Botijuela,-25.743034,-67.82325,S,40.00,6.44,NA,10.0,8.65,700.480,14643.0
AG22,ARG19,Argentina,Rosario de la Frontera,-25.409860,-64.59134,S,82.00,8.23,-8.28,0.0,1.57,151.800,3056.0
AG24,ARG19,Argentina,El Galpón,-24.409860,-64.59146,S,54.30,8.47,-10.67,0.0,1.81,94.662,3474.0
AG3,ARG19,Argentina,Pompeya,-24.246688,-66.36272,S,50.30,6.53,-5.33,10.0,5.09,930.309,9000.0


In [8]:
CAs <- phyloseq(
    otu_table(otu_table_final, taxa_are_rows = T),
    tax_table(tax_table),
    sample_data(sample_data_final)
)

CAs

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 4 taxa and 139 samples ]
sample_data() Sample Data:       [ 139 samples by 13 sample variables ]
tax_table()   Taxonomy Table:    [ 4 taxa by 2 taxonomic ranks ]

## alpha_diversity

In [ ]:
p.alph_div <- plot_richness(CAs, measures=c("shannon"), x="nation") + 

geom_boxplot(aes(fill=nation), lwd=0.3) +  

scale_fill_viridis(discrete=T) +

#scale_fill_manual(values = c("#B2182B", "#D6604D", "#F4A582", "#FDDBC7", "#D1E5F0"),
                  #labels=c("x","y","z")) +

#scale_x_discrete(name="",labels=c("x","y","z")) +

theme_glab() + theme(legend.title = element_blank())

p.alph_div

Warning message in estimate_richness(physeq, split = TRUE, measures = measures):
“The data you have provided does not have
any singletons. This is highly suspicious. Results of richness
estimates (for example) are probably unreliable, or wrong, if you have already
trimmed low-abundance taxa from the data.

We recommended that you find the un-trimmed data and retry.”


In [ ]:
data.frame(sample_data(CAs))

In [ ]:
CAs_alpha <- data.frame(
                        estimate_richness(CAs, measures = c("Shannon")),
                        data.frame(sample_data(CAs)$temperature),
                        data.frame(sample_data(CAs)$pH),
                        data.frame(sample_data(CAs)$salinity),
                        data.frame(sample_data(CAs)$nation)
)
CAs_alpha

In [ ]:
p.shan_temp <-ggplot(CAs_alpha,aes(x=sample_data.CAs..temperature,y=Shannon)) + 
geom_point(size=8,aes(fill=sample_data.CAs..nation),stroke=.3, shape=21)  +
scale_fill_viridis(discrete=TRUE) +
# scale_fill_manual(values=c("#440154","#3b528b","#2a788e","#fde725")) +
# geom_text(aes(label= sample_data.prok_ndata..code), size=5, hjust=-0.1, vjust=2.2) +
stat_poly_eq(formula = y ~ x, aes(label = paste(..rr.label.., sep = "~~")), parse = TRUE,hjust=-5,size=7) +
             geom_smooth(method=lm, formula= y~x,  se=FALSE,color="red",size=.4) +
xlab("pH") + 
guides(fill = guide_legend(override.aes = list(shape = 21) ),
            shape = guide_legend(override.aes = list(fill = "black"))) + 
labs(fill="This study", shape="Sample type") + 
ylim(NA,6.5) +
theme_glab()

p.shan_temp

In [ ]:
p.shan_temp <-ggplot(CAs_alpha,aes(x=sample_data.CAs..pH,y=Shannon)) + 
geom_point(size=8,aes(fill=sample_data.CAs..nation),stroke=.3, shape=21)  +
scale_fill_viridis(discrete=TRUE) +
# scale_fill_manual(values=c("#440154","#3b528b","#2a788e","#fde725")) +
# geom_text(aes(label= sample_data.prok_ndata..code), size=5, hjust=-0.1, vjust=2.2) +
stat_poly_eq(formula = y ~ x, aes(label = paste(..rr.label.., sep = "~~")), parse = TRUE,hjust=-5,size=7) +
             geom_smooth(method=lm, formula= y~x,  se=FALSE,color="red",size=.4) +
xlab("pH") + 
guides(fill = guide_legend(override.aes = list(shape = 21) ),
            shape = guide_legend(override.aes = list(fill = "black"))) + 
labs(fill="This study", shape="Sample type") + 
ylim(NA,6.5) +
theme_glab()

p.shan_temp

In [ ]:
plot_richness(CAs, measures=c("shannon"), x="as.factor(pH)") + 
geom_boxplot(aes(fill=pH),lwd=0.2) +  

scale_fill_viridis(discrete=F,option="plasma") + 


labs(x="") + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1),
                     axis.text=element_text(size=12,face="bold")) + theme_glab()

## BARPLOTS

In [ ]:
CAs_norm = transform_sample_counts(CAs, function(x){x / sum(x)})
CAs_norm

In [ ]:
CAs_hmm = tax_glom(CAs_norm, "hmm", NArm = FALSE)
CAs_hmm

In [ ]:
phyl_bio <- plot_bar(subset_samples(CAs_hmm), fill="hmm", x="nation", title = "") +  

#gghighlight(phyl_bio$data$Abundance > 0.009, use_group_by = FALSE) +

labs(x="",
     y="Relative Abundance (%)") +

theme_glab() + theme(legend.position = "right")+
theme(axis.text.x = element_text(angle = 0, vjust = 0.25, hjust=0.5),
      axis.text=element_text(size=10,face="bold")) 

phyl_bio

## Beta-Diversity analysis
## NMDS Jaccard similarity index: Weighted and Unweighted
## Weighted PCoA Jaccard

In [ ]:
CAs_wjak <- phyloseq::distance(CAs, method = "bray")
CAs_jw <- ordinate(CAs,CAs_wjak, method = "PCoA")
evals_jw <- CAs_jw$values$Eigenvalues

In [ ]:
p1 <- plot_ordination(CAs, CAs_jw, type="sample", title="PCoA weighted Jaccard similarity index") +
#coord_fixed(sqrt(evals_jw[2] / evals_jw[1]))  + 
geom_point(aes(fill=nation), size=5, shape=21, stroke=0.3) + 
#geom_text(aes(label= sample), size=4, hjust=0.2,vjust=2) + 
scale_fill_viridis(discrete=T) +
scale_shape_manual(values=c(21:23)) +
theme_glab() + theme(legend.position = "right")+ 
guides(fill = guide_legend(override.aes = list(shape = 21))) 
p1

In [ ]:
p1 <- plot_ordination(CAs, CAs_jw, type="sample", title="PCoA weighted Jaccard similarity index") +
#coord_fixed(sqrt(evals_jw[2] / evals_jw[1]))  + 
geom_point(aes(fill=pH), size=5, shape=21, stroke=0.3) + 
#geom_text(aes(label= sample), size=4, hjust=0.2,vjust=2) + 
scale_fill_viridis(discrete=F) +
scale_shape_manual(values=c(21:23)) +
theme_glab() + theme(legend.position = "right")+ 
guides(fill = guide_legend(override.aes = list(shape = 21))) 
p1

## nMDS weighted Jaccard

In [ ]:
plot_ordination(CAs, CAs_jw, type="taxa",title="nMDS weighted Jaccard similarity") +

    geom_point(aes(fill=CA_class),shape=21, size=6,color="black",stroke=0.3) + 

    #geom_text(aes(label= nation), size=4, hjust=0.2,vjust=2) +

    scale_fill_viridis(discrete=T) + 

    scale_shape_manual(values=c(21:24)) +

    theme_glab() + theme(legend.position = "right") +

    guides(fill = guide_legend(override.aes = list(shape = 21) ),
            shape = guide_legend(override.aes = list(fill = "black")))

## nMDS unweighted jaccard

In [ ]:
CAs_wjac <- phyloseq::distance(CAs, method = "jaccard")
CAs_unjac <- phyloseq::distance(CAs, method = "jaccard", binary = TRUE)

In [ ]:
CAs_juw <- ordinate(CAs, CAs_unjac, method = "NMDS",trymax=100)

In [ ]:
plot_ordination(CAs, CAs_juw, type="samples",title="nMDS unweighted Jaccard similarity") +

        geom_point(aes(fill=nation,shape=type),size=7,color="black",stroke=0.3) + 

        #geom_text(aes(label= nation), size=4, hjust=0.4,vjust=2) + 

        scale_fill_viridis(discrete=TRUE) + scale_shape_manual(values=c(21:24)) +

        theme_glab() + theme(legend.position = "right") +

        guides(fill = guide_legend(override.aes = list(shape = 21) ),
              shape = guide_legend(override.aes = list(fill = "black")))

In [ ]:
CAs_jw <- ordinate(CAs,CAs_wjac, method = "NMDS",trymax=100)

In [ ]:
plot_ordination(CAs, CAs_jw, type="samples",title="nMDS weighted Jaccard similarity") +

    geom_point(aes(fill=nation, shape=type),size=6,color="black",stroke=0.3) + 

    #geom_text(aes(label= nation), size=4, hjust=0.2,vjust=2) +

    scale_fill_viridis(discrete=T) + 

    scale_shape_manual(values=c(21:24)) +

    theme_glab() + theme(legend.position = "right") +

    guides(fill = guide_legend(override.aes = list(shape = 21) ),
            shape = guide_legend(override.aes = list(fill = "black")))

In [ ]:
plot_ordination(CAs, CAs_jw, type="samples",title="nMDS weighted Jaccard similarity") +

    geom_point(aes(fill=pH,shape=nation),size=6,color="black",stroke=0.3) + 

    #geom_text(aes(label= nation), size=4, hjust=0.2,vjust=2) +

    scale_fill_viridis(discrete=F) + 

    scale_shape_manual(values=c(21:25)) +

    theme_glab() + theme(legend.position = "right") +

    guides(fill = guide_legend(override.aes = list(shape = 21) ),
            shape = guide_legend(override.aes = list(fill = "black")))

## Vector Fitting Weighted Jaccard

In [ ]:
nmds_df1_jw<-cbind(sample_data(CAs),CAs_jw$points)
nmds_df1_jw

In [ ]:
nmds_df1.1_jw <- nmds_df1_jw[,4:15]

nmds_df1.1_jw <- nmds_df1.1_jw[ -c(3) ]

nmds_df1.1_jw


In [ ]:
message("Test with Pearson correlation vs NMDS1:")
# Pearson
for (i in 1:length(nmds_df1.1_jw)) {
    a <- cor.test(nmds_df1.1_jw[,i], nmds_df1.1_jw$MDS1)
       if (a$p.value<0.05) {
           print(paste(i,colnames(nmds_df1.1_jw)[i],a$estimate, a$parameter, a$p.value))
       }
}

message("Test with Pearson correlation vs NMDS2:")
# Pearson
for (i in 1:length(nmds_df1.1_jw)) {
    a <- cor.test(nmds_df1.1_jw[,i], nmds_df1.1_jw$MDS2)
       if (a$p.value<0.01) {
           print(paste(i,colnames(nmds_df1.1_jw)[i],a$estimate, a$parameter, a$p.value))
       }
}

In [ ]:
env_jw <-envfit(nmds_df1.1_jw[,c(10:11)], nmds_df1.1_jw[,c(1:9)], perm = 9999, na.rm = T)
env_jw

In [ ]:
env.scores_jw <- as.data.frame(scores(env_jw, display = "vectors"))         #extracts relevant scores from envifit
env.scores_jw <- cbind(env.scores_jw, env.variables = rownames(env.scores_jw)) #and then gives them their names

env.scores_jw <- cbind(env.scores_jw, pval = env_jw$vectors$pvals) # add pvalues to dataframe
sig.env.scrs_jw <- subset(env.scores_jw, pval<=0.15) #subset data to show variables significant at 0.05

sig.env.scrs_jw

In [ ]:
en_coord_cont.1_jw = sig.env.scrs_jw[,1:2] * ordiArrowMul(env_jw)
en_coord_cont.1_jw

In [ ]:
nmds_jw_envfit <- ggplot(data = nmds_df1_jw, aes(x = MDS1, y = MDS2)) + 
     
        geom_segment(data = en_coord_cont.1_jw, aes(x = 0, y = 0, xend = MDS1, yend = MDS2), 
                            size =.8, alpha = 0.8, colour = "grey") + 
     
        geom_text(data = en_coord_cont.1_jw, aes(x = MDS1, y = MDS2), colour = "grey30", 
                         fontface = "bold", size=6, label = row.names(en_coord_cont.1_jw)) + 
     
        geom_point(aes(fill = nation ,shape = type), size = 8) +
    
        scale_fill_viridis(discrete=TRUE) +
     
        scale_shape_manual(values = c(24,22,23,24,25)) + 
     
        # geom_text(aes(label = type), size=5, hjust=0.2, vjust=2) +
     
        theme(axis.title = element_text(size = 15, face = "bold", colour = "black"), 
                   panel.background = element_blank(), panel.border = element_rect(fill = NA, colour = "black"), 
                   axis.ticks = element_blank(), legend.key = element_blank(), 
                   legend.title = element_text(size = 10, face = "bold", colour = "grey30"), 
                   legend.text = element_text(size = 9, colour = "grey30")) + 
       
        xlab("NMDS1") + ylab("NMDS2") + 
       
        guides(fill = guide_legend(override.aes = list(shape = 22) ),
            shape = guide_legend(override.aes = list(fill = "black"))) + 
        
        theme_glab() + 

        theme(legend.position = "none")

nmds_jw_envfit